<h3>Iterators</h3>

In [1]:
class Squares:
    def __init__(self, length):
        self.length = length
        self.i = 0
        
    def __next__(self):
        if self.i >= self.length:
            raise StopIteration('Sequence is over')
        else:
            result = self.i ** 2
            self.i += 1
            return result
        
    def __iter__(self):               # iterator, returns the object itself
        return self

In [2]:
sq = Squares(3)

for item in sq:
    print(item)
    
# print(next(sq))  # error: StopIteration: Sequence is over

0
1
4


<br>

In [3]:
# let's look at the working mechanism

class Squares:
    def __init__(self, length):
        self.length = length
        self.i = 0
        
    def __next__(self):
        print('  __next__ called')
        if self.i >= self.length:
            raise StopIteration('Sequence is over')
        else:
            result = self.i ** 2
            self.i += 1
            return result
        
    def __iter__(self):               # iterator, returns the object itself
        print('  __iter__ called')
        return self

In [4]:
sq = Squares(3)

for item in sq:
    print(item)

  __iter__ called
  __next__ called
0
  __next__ called
1
  __next__ called
4
  __next__ called


<br>

from <a href="https://www.udemy.com/course/python-3-deep-dive-part-2/learn/lecture/10059700#questions/8297476">FAQ</a>:<br>
<code>sq</code> here is both an iterator and iterable.<br>
An iterator is an iterable since it implements <code>\_\_iter\_\_</code>

—<br>
from lecture "<a href="https://www.udemy.com/course/python-3-deep-dive-part-2/learn/lecture/10059702#questions/8302044">37. Iterators and iterables</a>":
- An <b>iterable</b> is an object that implements <code>\_\_iter\_\_</code> (<i>returns an iterator</i>)
- An <b>iterator</b> is an object that implements <code>\_\_iter\_\_</code> (<i>returns itself</i>) and <code>\_\_next\_\_</code>

<b>Iterators</b> are like iterables but become exhausted. <b>Iterables</b> on the other hand never become exhausted because they always return a new iterator that is then used to iterate.

<br>
<br>
<h3>Iterators and iterables</h3>

In [5]:
class Cities:
    def __init__(self):
        self._cities = ['Beijing', 'Seoul', 'Tokyo']
        self._index = 0
        
    def __len__(self):
        return len(self._cities)
    
    def __iter__(self):
        return CityIterator(self)

In [6]:
class CityIterator:
    def __init__(self, city_obj):
        self._city_obj = city_obj
        self._index = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._index >= len(self._city_obj):
            raise StopIteration
        else:
            item = self._city_obj._cities[self._index]
            self._index += 1
            return item

In [7]:
cities = Cities()

for city in cities:
    print(city)

Beijing
Seoul
Tokyo


In [8]:
# iterator is not exhausted
for city in cities:
    print(city)

Beijing
Seoul
Tokyo


<br>

In [9]:
# let's see what under the hood

class Cities:
    def __init__(self):
        self._cities = ['Beijing', 'Seoul', 'Tokyo']
        self._index = 0
        
    def __len__(self):
        return len(self._cities)
    
    def __iter__(self):
        print('Cities __iter__ called')
        return CityIterator(self)

In [10]:
class CityIterator:
    def __init__(self, city_obj):
        print('CityIterator new object!')
        self._city_obj = city_obj
        self._index = 0
        
    def __iter__(self):
        print('CityIterator __iter__ called')
        return self
    
    def __next__(self):
        print('CityIterator __next__ called')
        if self._index >= len(self._city_obj):
            raise StopIteration
        else:
            item = self._city_obj._cities[self._index]
            self._index += 1
            return item

In [11]:
cities = Cities()

for city in cities:
    print(city)

Cities __iter__ called
CityIterator new object!
CityIterator __next__ called
Beijing
CityIterator __next__ called
Seoul
CityIterator __next__ called
Tokyo
CityIterator __next__ called


<br>

In [12]:
# delete 'CityIterator' and 'Cities' so they don't confuse us
if 'CityIterator' in globals():
    del CityIterator
    
if 'Cities' in globals():   
    del Cities

<br>

In [13]:
# use nested class and add __getitem__

class Cities:
    def __init__(self):
        self._cities = ['Beijing', 'Seoul', 'Tokyo']
        self._index = 0
        
    def __len__(self):
        return len(self._cities)
    
    def __getitem__(self, s):            # for using constructions like cities[0]
        return self._cities[s]
    
    def __iter__(self):
        return self.CityIterator(self)   # attention to appearance of 'self'
     
    class CityIterator:
        def __init__(self, city_obj):
            self._city_obj = city_obj
            self._index = 0

        def __iter__(self):
            return self

        def __next__(self):
            if self._index >= len(self._city_obj):
                raise StopIteration
            else:
                item = self._city_obj._cities[self._index]
                self._index += 1
                return item

In [14]:
cities = Cities()

for city in cities:
    print(city)

Beijing
Seoul
Tokyo


In [15]:
print(cities[0])

Beijing


<br>
<br>
<h4>Consuming iterators manually</h4>
Task: read data about cars from csv file (with two first line of header) and write them in list 'cars'

In [16]:
# data casting
def cast(data_type, value):
    if data_type == 'DOUBLE':
        return float(value)
    elif data_type == 'INT':
        return int(value)
    else:
        return str(value)

In [17]:
def cast_row(data_types, data_row):
    return [cast(data_type, value) for data_type, value in zip(data_types, data_row)]

In [18]:
from collections import namedtuple
cars = []

with open('working_files/cars.csv') as fh:
    file_iter = iter(fh)                # actually here we can not use 'file_iter',
                                        # because 'fh' is itself iterable and iterator simultaneously
                                        # fh == file_iter -> true
    headers = next(file_iter).strip().split(';')
    Car = namedtuple('Car', headers)
    data_types = next(file_iter).strip().split(';')
    
    for line in file_iter:
        data = line.strip().split(';')
        data = cast_row(data_types, data)
        car = Car(*data)
        cars.append(car)


cars[:3]

[Car(Car='Chevrolet Chevelle Malibu', MPG=18.0, Cylinders=8, Displacement=307.0, Horsepower=130.0, Weight=3504.0, Acceleration=12.0, Model=70, Origin='US'),
 Car(Car='Buick Skylark 320', MPG=15.0, Cylinders=8, Displacement=350.0, Horsepower=165.0, Weight=3693.0, Acceleration=11.5, Model=70, Origin='US'),
 Car(Car='Plymouth Satellite', MPG=18.0, Cylinders=8, Displacement=318.0, Horsepower=150.0, Weight=3436.0, Acceleration=11.0, Model=70, Origin='US')]

In [19]:
# so without file_iter the program looks like this

from collections import namedtuple
cars = []

with open('working_files/cars.csv') as fh:
    headers = next(fh).strip().split(';')
    Car = namedtuple('Car', headers)
    data_types = next(fh).strip().split(';')
    
    for line in fh:
        data = line.strip().split(';')
        data = cast_row(data_types, data)
        car = Car(*data)
        cars.append(car)


cars[:3]

[Car(Car='Chevrolet Chevelle Malibu', MPG=18.0, Cylinders=8, Displacement=307.0, Horsepower=130.0, Weight=3504.0, Acceleration=12.0, Model=70, Origin='US'),
 Car(Car='Buick Skylark 320', MPG=15.0, Cylinders=8, Displacement=350.0, Horsepower=165.0, Weight=3693.0, Acceleration=11.5, Model=70, Origin='US'),
 Car(Car='Plymouth Satellite', MPG=18.0, Cylinders=8, Displacement=318.0, Horsepower=150.0, Weight=3436.0, Acceleration=11.0, Model=70, Origin='US')]

<br>
<br>
<h4>Cyclic iterators</h4>
Task: create iterator that constuct infinite sequence<br>

```
1 2 3 4 5 6 7 8 9 10 ...
N S W E

1N 2S 3W 4E 5N 6S 7W 8E 9N 10S ...
```

In [20]:
# infinite iterator
class CyclicIterator:
    def __init__(self, lst):
        self.lst = lst
        self.i = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        result = self.lst[self.i % len(self.lst)]
        self.i += 1
        return result

In [21]:
# let's see CyclicIterator at work
iter_cycl = CyclicIterator('NSWE')

for _ in range(6):
    print(next(iter_cycl))

N
S
W
E
N
S


In [22]:
n = 10
iter_cycl = CyclicIterator('NSWE')

for i in range(1, n+1):
    direction = next(iter_cycl)
    print(f'{i}{direction}')

1N
2S
3W
4E
5N
6S
7W
8E
9N
10S


In [23]:
# or using list comprehension
n = 10
iter_cycl = CyclicIterator('NSWE')

items = [f'{i}{next(iter_cycl)}' for i in range(1, n+1)]
items

['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']

In [24]:
# or using list comprehension with zip
n = 10
iter_cycl = CyclicIterator('NSWE')

items = [f'{number}{direction}' for number, direction in zip(range(1, n+1), iter_cycl)]
items

['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']

In [25]:
# a little differently
n = 10

items = [f'{number}{direction}' for number, direction in zip(range(1, n+1), 'NSWE' * (n // 4 + 1))]
items

['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']

In [26]:
# or using itertools.cycle
# [details - help(itertools.cycle)]

import itertools

n = 10
iter_cycl = itertools.cycle('NSWE')

items = [f'{i}{next(iter_cycl)}' for i in range(1, n+1)]
items

['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']

<br>
<br>
<h3>Lazy iterables</h3>

In [27]:
import math

In [28]:
class Circle:
    def __init__(self, r):
        self.radius = r
        self._area = None
        
    @property
    def radius(self):
        return self._radius
    
    @radius.setter
    def radius(self, r):
        self._radius = r
        self._area = None
    
    @property
    def area(self):
        if self._area is None:
            print('Calculating area...')
            self._area = math.pi * (self.radius**2)
        return self._area

In [29]:
c = Circle(1)

print(c.area)

Calculating area...
3.141592653589793


In [30]:
c.radius = 2

print(c.area)

Calculating area...
12.566370614359172


In [31]:
print(c.area)
# Since we already calculated the area and didn't change the radius after that, the area is not calculated again.

12.566370614359172


<br>
<br>
<hr>

In [32]:
# finite iterable
class Factorials:
    def __init__(self, length):
        self.length = length
        
    def __iter__(self):
        return self.FactIter(self.length)
    
    class FactIter:
        def __init__(self, length):
            self.length = length
            self.i = 0
            
        def __iter__(self):
            return self
        
        def __next__(self):
            if self.i >= self.length:
                raise StopIteration
            else:
                result = math.factorial(self.i)
                self.i += 1
                return result

In [33]:
facts = Factorials(6)

list(facts)

[1, 1, 2, 6, 24, 120]

<br>

In [34]:
# infinite iterable
class Factorials:
    def __iter__(self):
        return self.FactIter()
    
    class FactIter:
        def __init__(self):
            self.i = 0
            
        def __iter__(self):
            return self
        
        def __next__(self):
            result = math.factorial(self.i)
            self.i += 1
            return result

In [35]:
facts = Factorials()

fact_iter = iter(facts)

for _ in range(6):
    print(next(fact_iter))

1
1
2
6
24
120


<br>
<br>
<h3>Built-in iterators and iterables</h3>

Examination of some built-in functions of Python (<code>range</code>, <code>zip</code>, <code>open</code>, <code>enumerate</code>, <code>dictionary.keys</code>)

In [36]:
r = range(10)

print(r)                     #> range(0, 10) -the object use lazy evaluation
print(type(r))               #> <class 'range'>
print('__iter__' in dir(r))  #> True
print('__next__' in dir(r))  #> False - thus 'range' is not iterator, it is iterable
print(iter(r))               #> <range_iterator object at 0x…> - thus we can iterate over the object
print(r.__iter__())          #> --/--
print(iter(r) == r)          #> False

range(0, 10)
<class 'range'>
True
False
False


In [37]:
z = zip([1, 2, 3], 'abc')

print(z)                     #> <zip object at 0x7ff2d032c9c8> -the object use lazy evaluation
print(type(z))               #> <class 'zip'>
print('__iter__' in dir(z))  #> True
print('__next__' in dir(z))  #> True - thus 'range' is iterator
print(iter(z))               #> <zip object at 0x7ff2d032c9c8> - thus we can iterate over the object
print(z.__iter__())          #> --/--
print(iter(z) == z)         #> True
print('——')

print(list(z))               #> [(1, 'a'), (2, 'b'), (3, 'c')]
print(list(z))               #> []  - the object was exhausted

<class 'zip'>
True
True
True
——
[(1, 'a'), (2, 'b'), (3, 'c')]
[]


In [38]:
fh = open('working_files/cars.csv')

print(fh)                     #> <_io.TextIOWrapper name='working_files/cars.csv' mode='r' encoding='UTF-8'>
print(type(fh))               #> <class '_io.TextIOWrapper'>
print('__iter__' in dir(fh))  #> True
print('__next__' in dir(fh))  #> True - thus 'open' is iterator
print(iter(fh))               #> <_io.TextIOWrapper name='working_files/cars.csv' mode='r' encoding='UTF-8'>
print(fh.__iter__())          #> --/--
print(iter(fh) == fh)         #> True
print('——')

print(next(fh), end='')
print(fh.__next__(), end='')
print(fh.readline(), end='')

fh.close()

<_io.TextIOWrapper name='working_files/cars.csv' mode='r' encoding='UTF-8'>
<class '_io.TextIOWrapper'>
True
True
<_io.TextIOWrapper name='working_files/cars.csv' mode='r' encoding='UTF-8'>
<_io.TextIOWrapper name='working_files/cars.csv' mode='r' encoding='UTF-8'>
True
——
Car;MPG;Cylinders;Displacement;Horsepower;Weight;Acceleration;Model;Origin
STRING;DOUBLE;INT;DOUBLE;DOUBLE;DOUBLE;DOUBLE;INT;CAT
Chevrolet Chevelle Malibu;18.0;8;307.0;130.0;3504.;12.0;70;US


In [39]:
e = enumerate('Python rocks!')

print(e)                     #> <enumerate object at 0x7ff2d0300d80> -the object use lazy evaluation
print(type(e))               #> <class 'enumerate'>
print('__iter__' in dir(e))  #> True
print('__next__' in dir(e))  #> True - thus 'enumerate' is iterator
print(iter(e))               #> <zip object at 0x7ff2d0300d80> - thus we can iterate over the object
print(e.__iter__())          #> --/--
print(iter(e) == e)          #> True
print('——')

print(list(e))               #> [(0, 'P'), (1, 'y'), (2, 't'), …]
print(list(e))               #> []  - the object was exhausted

<class 'enumerate'>
True
True
True
——
[(0, 'P'), (1, 'y'), (2, 't'), (3, 'h'), (4, 'o'), (5, 'n'), (6, ' '), (7, 'r'), (8, 'o'), (9, 'c'), (10, 'k'), (11, 's'), (12, '!')]
[]


In [40]:
d = {'a': 1, 'b': 2}
keys = d.keys()
print(iter(keys) is keys)       #> False  - dictionary.keys() is iterable
print('__iter__' in dir(keys))  #> True
print('__next__' in dir(keys))  #> False

False
True
False


In [41]:
# The same with dictionaty.items and dictionary.values

<br>
<br>
<h3>Sorting iterables</h3>

In [42]:
import random

In [43]:
class RandomInts:
    def __init__(self, length, *, seed=0, lower=0, upper=10):
        self.length = length
        self.seed = seed
        self.lower = lower
        self.upper = upper
        
    def __len__(self):
        return self.length
    
    def __iter__(self):
        return self.RandomIterator(self.length,
                                   seed = self.seed,
                                   lower = self.lower,
                                   upper = self.upper)
    
    class RandomIterator:
        def __init__(self, length, *, seed, lower, upper):
            self.length = length
            self.lower = lower
            self.upper = upper
            self.num_requests = 0
            random.seed(seed)
            
        def __iter__(self):
            return self
        
        def __next__(self):
            if self.num_requests >= self.length:
                raise StopIteration
            else:
                result = random.randint(self.lower, self.upper)
                self.num_requests += 1
                return result

In [44]:
randoms = RandomInts(10)

for num in randoms:
    print(num, end=' ')
print()

for num in randoms:
    print(num, end=' ')

6 6 0 4 8 7 6 4 7 5 
6 6 0 4 8 7 6 4 7 5 

In [45]:
randoms = RandomInts(10, seed=None)

for num in randoms:
    print(num, end=' ')
print()

for num in randoms:
    print(num, end=' ')

6 6 2 4 4 2 10 9 0 3 
10 9 5 1 10 8 3 4 5 0 

In [46]:
randoms = RandomInts(10)

print(list(randoms))

print(sorted(randoms))

print(sorted(randoms, reverse=True))

[6, 6, 0, 4, 8, 7, 6, 4, 7, 5]
[0, 4, 4, 5, 6, 6, 6, 7, 7, 8]
[8, 7, 7, 6, 6, 6, 5, 4, 4, 0]


<br>

<br>
<br>
<h3>Detection whether the object is iterable</h3>

In [47]:
obj1 = 50
obj2 = [10, 20, 30]

In [48]:
def is_iterable(obj):
    try:
        iter(obj)
        return True
    except TypeError:
        return False
    
print(is_iterable(obj1))
print(is_iterable(obj2))

False
True


In [49]:
# alternative approach
def explore_obj(obj):
    try:
        for i in obj:
            print(i, end=" ")        
    except TypeError:
        print(f"Object '{obj}' is not iterable")
    
explore_obj(obj1)
explore_obj(obj2)

Object '50' is not iterable
10 20 30 

<br>
<br>
<h3>Iterating callables</h3>
Skipped. I consider that this topic is unpractical and simultaneously too complicated to keep summary.

<br>
<br>
<h3>Delegating iterators</h3>

In [50]:
from collections import namedtuple

Person = namedtuple('Person', 'first, last')

In [51]:
class PersonNames:
    def __init__(self, persons):
        try:
            self._persons = [person.first.capitalize() + ' ' + person.last.capitalize()
                             for person in persons]
        except (TypeError, AttributeError):
            self._persons = []
            
    def __iter__(self):
        return iter(self._persons)  # we delegate the responsibility for iteration to the list itself

In [52]:
persons = [Person('reNe', 'DESCarteS'),
           Person('robert', 'HOOKE'),
           Person('isAAc', 'newTON')]

person_names = PersonNames(persons)

for name in person_names:  # now it works
    print(name)

Rene Descartes
Robert Hooke
Isaac Newton


<br>
<br>
<h3>Reversed iteration</h3>

In [53]:
_SUITS = ('Spades', 'Hearts', 'Diamonds', 'Clubs')
_RANKS = tuple(range(2, 11)) + tuple('JQKA')        # (2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A')

In [54]:
from collections import namedtuple

Card = namedtuple('Card', 'rank, suit')

In [55]:
class CardDeck:
    def __init__(self):
        self.length = len(_SUITS) * len(_RANKS)
        
    def __len__(self):
        return self.length
    
    def __iter__(self):
        return self.CardDeckIterator(self.length)
    
    def __reversed__(self):                                     # reversed method     
        return self.CardDeckIterator(self.length, reverse=True)
    
    class CardDeckIterator:
        def __init__(self, length, reverse=False):
            self.length = length
            self.reverse = reverse
            self.i = 0
            
        def __iter__(self):
            return self
        
        def __next__(self):
            if self.i >= self.length:
                raise StopIteration
            else:
                if self.reverse:
                    index = self.length - 1 - self.i
                else:
                    index = self.i
                
                suit = _SUITS[index // len(_RANKS)]
                rank = _RANKS[index %  len(_RANKS)]
                self.i += 1
                return Card(rank, suit)

In [56]:
deck = reversed(CardDeck())

for card in deck:
    print(card)

Card(rank='A', suit='Clubs')
Card(rank='K', suit='Clubs')
Card(rank='Q', suit='Clubs')
Card(rank='J', suit='Clubs')
Card(rank=10, suit='Clubs')
Card(rank=9, suit='Clubs')
Card(rank=8, suit='Clubs')
Card(rank=7, suit='Clubs')
Card(rank=6, suit='Clubs')
Card(rank=5, suit='Clubs')
Card(rank=4, suit='Clubs')
Card(rank=3, suit='Clubs')
Card(rank=2, suit='Clubs')
Card(rank='A', suit='Diamonds')
Card(rank='K', suit='Diamonds')
Card(rank='Q', suit='Diamonds')
Card(rank='J', suit='Diamonds')
Card(rank=10, suit='Diamonds')
Card(rank=9, suit='Diamonds')
Card(rank=8, suit='Diamonds')
Card(rank=7, suit='Diamonds')
Card(rank=6, suit='Diamonds')
Card(rank=5, suit='Diamonds')
Card(rank=4, suit='Diamonds')
Card(rank=3, suit='Diamonds')
Card(rank=2, suit='Diamonds')
Card(rank='A', suit='Hearts')
Card(rank='K', suit='Hearts')
Card(rank='Q', suit='Hearts')
Card(rank='J', suit='Hearts')
Card(rank=10, suit='Hearts')
Card(rank=9, suit='Hearts')
Card(rank=8, suit='Hearts')
Card(rank=7, suit='Hearts')
Card(ran

<br>
<h4>Reversed iteration</h4>

In [57]:
class Squares:
    def __init__(self, length):
        self.squares = [i **2 for i in range(length)]
        
    def __len__(self):
        return len(self.squares)
    
    def __getitem__(self, s):
        return self.squares[s]

In [58]:
# If sequence implements __len__ and __getitem__ then Python can implement 'reversed' itself
for num in reversed(Squares(5)):
    print(num)

16
9
4
1
0


In [59]:
# but we also can implement __reversed__ manually

<br>
<br>
<h3>Using iterators as function arguments</h3>

In [60]:
import random

In [61]:
class Randoms:
    def __init__(self, n):
        self.n = n
        self.i = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        else:
            self.i += 1
            return random.randint(0, 100)

In [62]:
random.seed(0)
rs = Randoms(10)
print(rs)
print(min(rs))
# print(max(rs))   # ValueError: max() arg is an empty sequence

5
